In [17]:

import torch
from torch import nn
from d2l import torch as d2l



In [18]:
# 卷积操作
def corr2d(X, K):  #@save
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

'''
 0 1 2 
 3 4 5  
 6 7 8
 
 0 1
 2 3
'''
# 在卷积层中，输入张量和核张量通过(互相关运算)产生输出张量。
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
# 输入 张量X 和卷积核张量 K
corr2d(X, K)


tensor([[19., 25.],
        [37., 43.]])

In [19]:
## 卷积层

# 卷积层对输入和卷积核权重进行互相关运算，并在添加标量偏置之后产生输出。
# 所以，卷积层中的两个被训练的参数是卷积核权重和标量偏置。
# 就像我们之前随机初始化全连接层一样，在训练基于卷积层的模型时，我们也随机初始化卷积核权重。 

class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias
    


In [20]:
# 一个简单的应用 通过CNN来找到像素变化的位置

# 6行8列的矩阵 一个图像其中 中间四列为黑色（$0$），其余像素为白色（$1$）。
X = torch.ones((6, 8))
X[:, 2:6] = 0
X


tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [21]:
# 构造卷积核 1*2
K = torch.tensor([[1.0, -1.0]])
K

tensor([[ 1., -1.]])

In [22]:
# 我们对参数X（输入）和K（卷积核）执行互相关运算。 如下所示，[输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘]，其他情况的输出为
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [23]:
# 现在我们将输入的二维图像转置，再进行如上的互相关运算。 其输出如下，之前检测到的垂直边缘消失了。 不出所料，这个[卷积核K只可以检测垂直边缘]，无法检测水平边缘。
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [24]:
## 卷积核的学习
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
# 注意观察 X 的形状
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率


for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')


epoch 2, loss 6.564
epoch 4, loss 1.927
epoch 6, loss 0.661
epoch 8, loss 0.249
epoch 10, loss 0.099


In [25]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0228, -0.9588]])